# Applied Data Science Capstone

## Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week1-Part 2)
Yader Rafael Carrillo Jaime

Future Physicist and Junior Data Scientist

## 1) Introduction/Business Problem
Around the world, hundreds of people are trying everyday to open small and medium businesses. No matter in what city they are planing to do it, they will look for the best place with the aim of increase their earnings. The present project, is directed to help future entrepreneurs in order to chose the best location to build their businesses in New York City, through providing data about neighborhoods' characteristics and common venues to set up the venture.

It should be noted that to reach this goal, we need to follow a particular structure to show the results. In this case, we were claimed to follow the typical Data science methodology. I hope to do the best of myself along the project.

## 2) Data
To reach the goal of this project and provide information to skateholders, I'll be using New York data and Foursquare API to extract competitors on the same neighborhoods.

New York data can be found here https://geo.nyu.edu/catalog/nyu_2451_34572

Before we can get the data, we have to download the libraries to use in this project

In [45]:
import numpy as np # we will handle data in a vectorized manner

import pandas as pd # we'll use it for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

import json 
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 2.1) Download and Explore Dataset
Let's download and explore dataset, and convert it into pandas dataframe

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods_data = newyork_data['features']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

We'll focus on Bronx. A very popular borough in New York City

In [7]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Before we continue exploring the data, we'll use Foursquare API to explore the neighborhoods venues

In [8]:
CLIENT_ID = 'VQNHUJ4ZBN4I5BU3BZCN22MPTSONQIZCNHMN41YY0K0DN5FC' # my Foursquare ID
CLIENT_SECRET = '1S3K2B0QGJEET24XEXM2VQZV2SG41XMKHK3Q24VBETVKNET4' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VQNHUJ4ZBN4I5BU3BZCN22MPTSONQIZCNHMN41YY0K0DN5FC
CLIENT_SECRET:1S3K2B0QGJEET24XEXM2VQZV2SG41XMKHK3Q24VBETVKNET4


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.85048545, -73.8404035580209.


In [11]:
LIMIT = 100
radius=500

bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


Now, we can check the size of our dataframe

In [12]:
print(bronx_venues.shape)
bronx_venues.head()

(1238, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


In [33]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,35,35,35,35,35,35
Baychester,20,20,20,20,20,20
Bedford Park,34,34,34,34,34,34
Belmont,99,99,99,99,99,99
Bronxdale,13,13,13,13,13,13
Castle Hill,8,8,8,8,8,8
City Island,24,24,24,24,24,24
Claremont Village,16,16,16,16,16,16
Clason Point,10,10,10,10,10,10


In [34]:
print('There are {} uniques categories.'.format(len(bronx_venues['Venue Category'].unique())))

There are 175 uniques categories.


In [35]:
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot =bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,Airport Tram,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nightclub,Office,Optical Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [36]:
bronx_onehot.shape

(1238, 176)

Grouping Bronx

In [37]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,Airport Tram,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nightclub,Office,Optical Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store,Yoga Studio
0,Allerton,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.028571,0.0000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085714,0.000000,0.000000,0.00,0.000000,0.028571,0.000000,0.028571,0.0000,0.00,0.057143,0.000000,0.028571,0.000000,0.028571,0.0000,0.00000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.028571,0.0000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,0.000000,0.114286,0.00,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.085714,0.028571,0.000000,0.000000,0.000000,0.000000,0.085714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Bayche

In [38]:
bronx_grouped.shape

(52, 176)

Most Common Venues

In [39]:
num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                venue  freq
0         Pizza Place  0.11
1                 Spa  0.09
2         Supermarket  0.09
3              Bakery  0.09
4  Chinese Restaurant  0.09


----Baychester----
                 venue  freq
0       Baseball Field  0.05
1    Convenience Store  0.05
2          Bus Station  0.05
3  Fried Chicken Joint  0.05
4           Donut Shop  0.05


----Bedford Park----
                 venue  freq
0                Diner  0.12
1        Deli / Bodega  0.09
2          Pizza Place  0.09
3   Mexican Restaurant  0.06
4  Fried Chicken Joint  0.06


----Belmont----
                venue  freq
0  Italian Restaurant  0.18
1         Pizza Place  0.09
2       Deli / Bodega  0.08
3              Bakery  0.05
4        Liquor Store  0.03


----Bronxdale----
                           venue  freq
0             Italian Restaurant  0.15
1  Paper / Office Supplies Store  0.08
2                 Breakfast Spot  0.08
3                    Supermarket  0.08
4             Mexican 

                venue  freq
0  Italian Restaurant   0.1
1         Pizza Place   0.1
2                 Bar   0.1
3         Coffee Shop   0.1
4        Liquor Store   0.1


----Unionport----
                       venue  freq
0  Latin American Restaurant  0.10
1                 Donut Shop  0.10
2             Ice Cream Shop  0.05
3         Spanish Restaurant  0.05
4          Indian Restaurant  0.05


----University Heights----
                 venue  freq
0          Pizza Place  0.08
1               Bakery  0.08
2   Chinese Restaurant  0.08
3  Fried Chicken Joint  0.08
4          Bus Station  0.04


----Van Nest----
            venue  freq
0   Deli / Bodega  0.32
1     Pizza Place  0.21
2          Bakery  0.11
3     Bus Station  0.05
4  Cosmetics Shop  0.05


----Wakefield----
                  venue  freq
0        Ice Cream Shop   0.1
1            Laundromat   0.1
2                  Food   0.1
3           Gas Station   0.1
4  Caribbean Restaurant   0.1


----West Farms----
         venue 

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Spa,Bakery,Supermarket,Chinese Restaurant,Deli / Bodega,Pharmacy,Bus Station,Spanish Restaurant,Breakfast Spot
1,Baychester,Breakfast Spot,Donut Shop,Mexican Restaurant,Fast Food Restaurant,Spanish Restaurant,Sandwich Place,Fried Chicken Joint,Supermarket,Electronics Store,Baseball Field
2,Bedford Park,Diner,Pizza Place,Deli / Bodega,Sandwich Place,Food Truck,Supermarket,Fried Chicken Joint,Chinese Restaurant,Mexican Restaurant,Train Station
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Grocery Store,Liquor Store,Dessert Shop,Bank,Sandwich Place,Spanish Restaurant
4,Bronxdale,Italian Restaurant,Spanish Restaurant,Mexican Restaurant,Supermarket,Eastern European Restaurant,Bank,Chinese Restaurant,Gym,Breakfast Spot,Pizza Place


In [42]:
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 1, 1, 4, 4, 2, 2, 2], dtype=int32)

In [43]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,Ice Cream Shop,Sandwich Place,Food Truck,Gas Station,Laundromat,Caribbean Restaurant,Donut Shop,Food,Pharmacy,Dessert Shop
1,Bronx,Co-op City,40.874294,-73.829939,2,Park,Bus Station,Fast Food Restaurant,Ice Cream Shop,Grocery Store,Pharmacy,Restaurant,Chinese Restaurant,Discount Store,Baseball Field
2,Bronx,Eastchester,40.887556,-73.827806,2,Bus Station,Caribbean Restaurant,Metro Station,Diner,Bus Stop,Deli / Bodega,Bowling Alley,Cosmetics Shop,Pizza Place,Convenience Store
3,Bronx,Fieldston,40.895437,-73.905643,0,Plaza,Bus Station,River,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor,Electronics Store
4,Bronx,Riverdale,40.890834,-73.912585,2,Bus Station,Park,Playground,Bank,Home Service,Plaza,Food Truck,Gym,Baseball Field,Fast Food Restaurant


Displaying Clusters on a Map

In [46]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [47]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Bus Station,River,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor,Electronics Store


Cluster 2

In [48]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Pelham Parkway,Pizza Place,Frozen Yogurt Shop,Italian Restaurant,Chinese Restaurant,Ice Cream Shop,Gas Station,Spa,Eye Doctor,Food,Sandwich Place
27,Throgs Neck,Deli / Bodega,Sports Bar,Liquor Store,Coffee Shop,Juice Bar,Asian Restaurant,Italian Restaurant,American Restaurant,Pizza Place,Bar
31,Van Nest,Deli / Bodega,Pizza Place,Bakery,Coffee Shop,Bus Station,Supermarket,Donut Shop,Middle Eastern Restaurant,Hookah Bar,Cosmetics Shop
32,Morris Park,Pizza Place,Deli / Bodega,Bakery,Burger Joint,Pharmacy,Donut Shop,Coffee Shop,Sandwich Place,Mexican Restaurant,Supermarket
33,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Grocery Store,Liquor Store,Dessert Shop,Bank,Sandwich Place,Spanish Restaurant
35,North Riverdale,Pizza Place,Deli / Bodega,Italian Restaurant,Bank,Coffee Shop,Pool,Chinese Restaurant,Donut Shop,Sandwich Place,Bus Station
36,Pelham Bay,Italian Restaurant,Deli / Bodega,Cosmetics Shop,Convenience Store,Gym / Fitness Center,Bank,Fast Food Restaurant,Donut Shop,Sandwich Place,Salon / Barbershop
38,Edgewater Park,Italian Restaurant,Coffee Shop,Deli / Bodega,Pizza Place,American Restaurant,Park,Spa,Bar,Asian Restaurant,Pub
49,Bronxdale,Italian Restaurant,Spanish Restaurant,Mexican Restaurant,Supermarket,Eastern European Restaurant,Bank,Chinese Restaurant,Gym,Breakfast Spot,Pizza Place


Cluster 3

In [49]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Park,Bus Station,Fast Food Restaurant,Ice Cream Shop,Grocery Store,Pharmacy,Restaurant,Chinese Restaurant,Discount Store,Baseball Field
2,Eastchester,Bus Station,Caribbean Restaurant,Metro Station,Diner,Bus Stop,Deli / Bodega,Bowling Alley,Cosmetics Shop,Pizza Place,Convenience Store
4,Riverdale,Bus Station,Park,Playground,Bank,Home Service,Plaza,Food Truck,Gym,Baseball Field,Fast Food Restaurant
14,Morris Heights,Food Truck,Spanish Restaurant,Pizza Place,Pharmacy,Bus Station,Recreation Center,Clothing Store,Grocery Store,Chinese Restaurant,Bank
17,West Farms,Bus Station,Hostel,Basketball Court,Park,Diner,Convenience Store,Playground,Coffee Shop,Donut Shop,Sandwich Place
25,Soundview,Chinese Restaurant,Spanish Restaurant,Grocery Store,Playground,Discount Store,Bus Stop,Bus Station,Breakfast Spot,Fried Chicken Joint,Basketball Court
26,Clason Point,Park,South American Restaurant,Scenic Lookout,Boat or Ferry,Pool,Bus Stop,Grocery Store,Fast Food Restaurant,Farmers Market,Eye Doctor
44,Edenwald,Grocery Store,Fish Market,Bus Station,Gas Station,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor,Electronics Store
45,Claremont Village,Bus Station,Bakery,Chinese Restaurant,Pizza Place,Gym,Supermarket,Caribbean Restaurant,Liquor Store,Grocery Store,Gift Shop


Cluster 4

In [50]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Williamsbridge,Caribbean Restaurant,Soup Place,Bar,Nightclub,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


Cluster 5

In [52]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Ice Cream Shop,Sandwich Place,Food Truck,Gas Station,Laundromat,Caribbean Restaurant,Donut Shop,Food,Pharmacy,Dessert Shop
5,Kingsbridge,Pizza Place,Sandwich Place,Discount Store,Supermarket,Bar,Mexican Restaurant,Bakery,Latin American Restaurant,Liquor Store,Donut Shop
6,Woodlawn,Pizza Place,Deli / Bodega,Playground,Park,Plaza,Donut Shop,Cosmetics Shop,Moving Target,Liquor Store,Food & Drink Shop
7,Norwood,Pizza Place,Park,Deli / Bodega,Chinese Restaurant,Pharmacy,Bank,Fast Food Restaurant,Coffee Shop,Supermarket,Mobile Phone Shop
9,Baychester,Breakfast Spot,Donut Shop,Mexican Restaurant,Fast Food Restaurant,Spanish Restaurant,Sandwich Place,Fried Chicken Joint,Supermarket,Electronics Store,Baseball Field
11,City Island,Harbor / Marina,Ice Cream Shop,Seafood Restaurant,Thrift / Vintage Store,Bank,Italian Restaurant,French Restaurant,Liquor Store,Music Venue,Diner
12,Bedford Park,Diner,Pizza Place,Deli / Bodega,Sandwich Place,Food Truck,Supermarket,Fried Chicken Joint,Chinese Restaurant,Mexican Restaurant,Train Station
13,University Heights,Bakery,Chinese Restaurant,Fried Chicken Joint,Pizza Place,Bus Station,Food,Sandwich Place,Laundromat,Supermarket,Latin American Restaurant
15,Fordham,Shoe Store,Mobile Phone Shop,Fast Food Restaurant,Donut Shop,Gym / Fitness Center,Spanish Restaurant,Pizza Place,Supplement Shop,Pharmacy,Bank
16,East Tremont,Pizza Place,Café,Deli / Bodega,Fast Food Restaurant,Smoke Shop,Breakfast Spot,Shoe Store,Lounge,Bus Station,Supermarket
